<a href="https://colab.research.google.com/github/jeevankumar20/INFO_5731_Spring2023/blob/main/Jeevan_In_class_exercise_02_02072023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The second In-class-exercise (02/07/2023, 40 points in total)

The purpose of this exercise is to understand users' information needs, then collect data from different sources for analysis.

Question 1 (10 points): Describe an interesting research question (or practical question) you have in mind, what kind of data should be collected to answer the question(s)? How many data needed for the analysis? The detail steps for collecting and save the data. 

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:

Some research questions that can be answered by the the bellow collected can be:

1. What are the most highly rated movies of all time according to IMDb users?
2. How has the average movie rating changed over time?
3. Do highly rated movies tend to have higher gross revenue?
4. Do movies with a specific rating certificate (e.g. G, PG, R) tend to receive higher ratings on IMDb?

>The data is collected from IMDb website and the specific data I collected is:
 Movie title, year of release, IMDb rating, runtime, Certificate(G, PG, R), and gross revenue.
>Web scraping is performed using a BeautifulSoup library to extract the data from the website.
>The data is loaded into a dataframe and is saved to a csv file.s

'''

Question 2 (10 points): Write python code to collect 1000 data samples you discussed above.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Request the page
url = "https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&ref_=adv_prv"
response = requests.get(url)

# Use BeautifulSoup to parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")

# Find all movie containers
movie_containers = soup.find_all("div", class_="lister-item mode-advanced")

# Create a list to store the movie information
movies = []

# Loop through each page
for page in range(1, 11):
    # Calculate the starting point of the movies for the current page
    start = (page - 1) * 100 + 1

    # Create the URL for the current page
    url = f"https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start={start}&ref_=adv_nxt"

    # Loop through each movie container and extract the movie information
    for container in movie_containers:
        name = container.h3.a.text
        year = container.h3.find("span", class_="lister-item-year").text
        rating = container.strong.text
        runtime = container.find("span", class_="runtime").text.strip()
        movie_type = container.find("span", class_="certificate").text.strip() if container.find("span", class_="certificate") else None
        gross_element = container.find("span", attrs={"name": "nv"})
        gross = gross_element["data-value"] if gross_element else None

    # Add the movie information to the list
        movies.append([name, year, rating, runtime, movie_type, gross])

# Convert the list of movies to a Pandas DataFrame
df = pd.DataFrame(movies, columns=["Movie", "Year", "Rating", "Runtime", "Certificate", "Gross"])

# Print the DataFrame
print(df)

df.to_csv("movies.csv", index=False)



                                     Movie    Year Rating  Runtime  \
0                 The Shawshank Redemption  (1994)    9.3  142 min   
1                            The Godfather  (1972)    9.2  175 min   
2                          The Dark Knight  (2008)    9.0  152 min   
3                         Schindler's List  (1993)    9.0  195 min   
4                    The Godfather Part II  (1974)    9.0  202 min   
..                                     ...     ...    ...      ...   
995                      Good Will Hunting  (1997)    8.3  126 min   
996                   Inglourious Basterds  (2009)    8.3  153 min   
997                                   Heat  (1995)    8.3  170 min   
998                    Requiem for a Dream  (2000)    8.3  102 min   
999  Eternal Sunshine of the Spotless Mind  (2004)    8.3  108 min   

    Certificate    Gross  
0             R  2698695  
1             R  1872940  
2         PG-13  2672462  
3             R  1364329  
4             R  1280001

Question 3 (10 points): Write python code to collect 1000 articles from Google Scholar (https://scholar.google.com/), Microsoft Academic (https://academic.microsoft.com/home), or CiteSeerX (https://citeseerx.ist.psu.edu/index), or Semantic Scholar (https://www.semanticscholar.org/), or ACM Digital Libraries (https://dl.acm.org/) with the keyword "information retrieval". The articles should be published in the last 10 years (2012-2022).

The following information of the article needs to be collected:

(1) Title

(2) Venue/journal/conference being published

(3) Year

(4) Authors

(5) Abstract

In [2]:
import requests
import pandas as pd

def collect_articles(query, num_articles, start_year, end_year):
    articles = []
    for i in range(0, num_articles, 100):
        url = "https://api.semanticscholar.org/graph/v1/paper/search?query={}&offset={}&limit=100&year={}-{}&fields=title,year,venue,journal,publicationTypes,authors,abstract".format(query, i, start_year, end_year)
        response = requests.get(url)
        results = response.json()
        if 'error' in results:
            print("Results: ", results)
            break
        for result in results['data']:
            article = {}
            article['title'] = result.get('title', '')
            article['year'] = result.get('year', '')
            article['venue'] = result.get('venue', '')
            article['journal'] = result.get('journal', '')
            article['publicationTypes'] = result.get('publicationTypes', [])
            article['authors'] = result.get('authors', [])
            article['abstract'] = result.get('abstract', '')
            articles.append(article)
    return articles

query = "information retrieval"
num_articles = 1000
start_year = 2012
end_year = 2022
articles = collect_articles(query, num_articles, start_year, end_year)

df = pd.DataFrame(articles)
print(df.head())
print(df.tail())





                                               title  year  \
0  BEIR: A Heterogenous Benchmark for Zero-shot E...  2021   
1                              Information Retrieval  2018   
2  COIL: Revisit Exact Lexical Match in Informati...  2021   
3  Pyserini: A Python Toolkit for Reproducible In...  2021   
4  SPLADE v2: Sparse Lexical and Expansion Model ...  2021   

                                               venue  \
0                    NeurIPS Datasets and Benchmarks   
1                  Lecture Notes in Computer Science   
2  North American Chapter of the Association for ...   
3  Annual International ACM SIGIR Conference on R...   
4                                              ArXiv   

                                             journal  \
0      {'name': 'ArXiv', 'volume': 'abs/2104.08663'}   
1  {'name': 'Canadian Medical Association journal...   
2                             {'pages': '3030-3042'}   
3  {'name': 'Proceedings of the 44th Internationa...   
4      {'n

Question 4 (10 points): Write python code to collect 1000 posts from Twitter, or Facebook, or Instagram. You can either use hashtags, keywords, user_name, user_id, or other information to collect the data. 

The following information needs to be collected:

(1) User_name

(2) Posted time

(3) Text 

In [11]:
# You code here (Please add comments in the code):
import tweepy

# Twitter API credentials
consumer_key = "u7L1lnR7HN85dn1qnTFO1cegb"
consumer_secret = "QN1JrEmit2To46ZcwWAT4aI5QGWZXWRDDUPnMCWV5M66SFc8wT"
access_token = "1144377060036620294-BSEicX3zH7hIhksbNZV9mrWFwa07cO"
access_token_secret = "gxWMOodDq1nQAjix9mHEOUSAtgE7XH5ctHInm0XRslJce"


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create the API object
api = tweepy.API(auth)

# Search for tweets containing the keyword "information retrieval"
tweets = tweepy.Cursor(api.search, q='covid', tweet_mode='extended').items(1000)

# Collect the relevant information
tweet_data = []
for tweet in tweets:
    user_name = tweet.user.screen_name
    posted_time = tweet.created_at
    text = tweet.full_text
    
    tweet_data.append([user_name, posted_time, text])

# Convert the tweet data to a pandas DataFrame

df = pd.DataFrame(tweet_data, columns=['User_name', 'Posted_time', 'Text'])


print(df)






           User_name         Posted_time  \
0      OlDer43624126 2023-02-10 03:30:20   
1           dillon66 2023-02-10 03:30:20   
2        grey_sister 2023-02-10 03:30:19   
3    DeiberthMartin1 2023-02-10 03:30:19   
4     GalloAnonymous 2023-02-10 03:30:19   
..               ...                 ...   
995         chaeisl_ 2023-02-10 03:25:16   
996  PankajSaini0220 2023-02-10 03:25:16   
997        bernKleen 2023-02-10 03:25:16   
998    notalemming24 2023-02-10 03:25:16   
999          averiyt 2023-02-10 03:25:16   

                                                  Text  
0    RT @Mippcivzla: 🗣️ ¡𝐄𝐒 𝐂𝐎𝐍𝐓𝐈𝐆𝐎!✍️🇻🇪\nEl síndro...  
1    RT @KanekoaTheGreat: FLASHBACK: YouTube censor...  
2    RT @IsisWise: On February 26, 2020 Heavoil Tec...  
3    RT @Mippcivzla: 🗣️ ¡𝐓𝐎𝐌𝐀 𝐏𝐑𝐄𝐂𝐀𝐔𝐂𝐈𝐎́𝐍!✍️🇻🇪\nVen...  
4    RT @ALEGRA1988: "Las *v☠cunas*  Covid potencia...  
..                                                 ...  
995  RT @purpletwts: @touringdata @JYPETWICE $33.9M...  
996  RT @DDIndi